In [160]:
from sagemaker.huggingface import HuggingFaceModel
import sagemaker

role = sagemaker.get_execution_role()
import pandas as pd

In [161]:
# Hub Model configuration. https://huggingface.co/models
# https://huggingface.co/MoritzLaurer/mDeBERTa-v3-base-mnli-xnli
hub = {
	'HF_MODEL_ID':'MoritzLaurer/mDeBERTa-v3-base-mnli-xnli',
	'HF_TASK':'zero-shot-classification'
}

# Create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
	transformers_version='4.6.1',
	pytorch_version='1.7.1',
	py_version='py36',
	env=hub,
	role=role, 
)

# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
	initial_instance_count=1, # number of instances
	instance_type='ml.p2.xlarge' # ec2 instance type - check the EC2 type capacity before execution
)


---------!

In [163]:
# ##load class
# text = pd.read_excel('AI对话活动.xlsx',engine='openpyxl')
# candidate_labels = list(text['Unnamed: 1'])
# #fiter nan
# candidate_labels = [0 if str(x)=='nan' else x for x in candidate_labels]

# JM - Test the model for the CX
candidate_labels = ["正向", "负向", "中立"]
print("CANDIDATE", candidate_labels)

In [175]:
%%time

def cls(input):
    data = {
        "inputs": input,
        "parameters": {
            "candidate_labels": candidate_labels,
            "hypothesis_template": "这个句子表示{}。"
        }
    }
    res = predictor.predict(data)
    labels = res['labels'][0]
    print ("input sentence ##{}##, class ##{}##, result detail {}".format(input,labels,res))
    print ("input sentence ##{}##, class ##{}##".format(input,labels))
    return 

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 8.58 µs


In [174]:
inputs = [
            "明天会是晴天",
            "明天会很冷",
            "我准备去旅游",
            "我在路上受伤了",
            "主播真漂亮",
            "主播666",
            "我要刷大火箭",
            "主播五音不全"
            "踩踩踩",
            "明天天气很热"
]

for i in inputs:
    cls(i)

input sentence ##I'm in a bad mood today##, class ##紧张##
input sentence ##i want to watch fast and furious 9, show me an appropriate lighting effects##, class ##愤怒##
input sentence ##I want to listen to Jay Chou's Mojito##, class ##音乐##
input sentence ##My favorite thing to eat is an omelet##, class ##美食##
input sentence ##Mother's Day is coming, I want to give her a present##, class ##节日##


In [176]:
%%time
### cpu CPU times: user 6.1 ms, sys: 0 ns, total: 6.1 ms Wall time: 2.61 s
inputs = [ "I'm in a bad mood today"]

for i in inputs:
    cls(i)

input sentence ##I'm in a bad mood today##, class ##紧张##, result detail {'sequence': "I'm in a bad mood today", 'labels': ['紧张', '担忧', '悲伤', '愤怒', '恐惧', 0, 0, 0, 0, 0, '唤醒', '助眠', '动物', '民族', '国家', '文化', '宇宙', '运动', '历史', '建筑', '季节', '汽车', '艺术', '森林', '风景', '教育', '电视剧', '海洋', '哲学', '游戏', '电影', '音乐', '节日', '天气', '绘画', '激情', '旅行', '聚会', '美食', '兴奋', '快乐'], 'scores': [0.23698003590106964, 0.10469383746385574, 0.04708351194858551, 0.03948110342025757, 0.03617393970489502, 0.02747998759150505, 0.02747998759150505, 0.02747998759150505, 0.02747998759150505, 0.02747998759150505, 0.025057367980480194, 0.022118063643574715, 0.0203788373619318, 0.018999870866537094, 0.018522758036851883, 0.01840215176343918, 0.017946835607290268, 0.0171367060393095, 0.01710280030965805, 0.01670113578438759, 0.016263023018836975, 0.01548792328685522, 0.015042421407997608, 0.013737933710217476, 0.01355113834142685, 0.013439377769827843, 0.012115479446947575, 0.011988149955868721, 0.01168679166585207, 0.0112150041386